In [2]:
import re
import openai
import time
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

class Writer():
    @staticmethod
    def ask_gpt3(prompt, max_retries=3, delay=2):
        retries = 0
        while retries < max_retries:
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=prompt,
                )
                assistant_message = {
                    "role": "assistant",
                    "content": response.choices[0].message["content"]
                }
                prompt.append(assistant_message)
                return assistant_message["content"]
            except openai.error.OpenAIError as e:
                print(f"Error occurred during API call: {e}")
                retries += 1
                if retries < max_retries:
                    time.sleep(delay)  # Delay between retries
                else:
                    raise

    def get_chars(self, input):
        messages = [{"role": "system", "content": "To solve this input, we need a list of characters and descriptions, both major and minor, who will have key parts to play. Help me write this please."}]
        user_message = {"role": "user", "content": input}
        messages.append(user_message)
        chars = self.ask_gpt3(messages)
        return chars

    def get_story(self, input, chars):
        messages = [
            {"role": "system",
                "content": "From the given prompt, generate a 6-part structure for a story, where each part has a 1 sentence description of what exactly happens in that part. Please start each part with Part X:"},
            {"role": "user", "content": input},
            {"role": "user", "content": f"The key characters for the overall story are {chars}"}
        ]
        response = self.ask_gpt3(messages)
        storyline_parts = re.split(r'Part \d+:', response)[1:]
        return [part.strip() for part in storyline_parts]

    def write_chapters(self, chars, storyline_parts):
        with open('story.txt', 'w', encoding='utf-8') as story_file:
            for index, part in enumerate(storyline_parts):
                print(f"{index}: {part}")
            for index, chapter_summary in enumerate(storyline_parts):
                messages = [{"role": "system", "content": "For this chapter, write the story and dialogue to explore it fully and bring the chapter to life"}]
                user_message = {"role": "user", "content": chapter_summary}
                char_message = {"role": "user", "content": f"The key characters for the overall story are {chars}"}
                messages.append(user_message)
                messages.append(char_message)
                chapter_content = self.ask_gpt3(messages)

                # Save chapter to file
                story_file.write(f'Chapter {index + 1}\n\n')
                story_file.write(chapter_content)
                story_file.write('\n\n')

if __name__ == "__main__":
    writer = Writer()
    input_text = "Write a short story in the style of Haruki Murakami about a machine just trying to live life in a complex world."
    chars = writer.get_chars(input_text)
    storyline = writer.get_story(input_text, chars)
    writer.write_chapters(chars, storyline)
    print("Story has been written to 'story.txt'")

0: The Machine's Existence
In a chaotic city, a nameless machine made of gears and wires is confused about its purpose and existence.
1: Observing Life Around
The machine observes the people around it and wonders why it feels so empty, watching the people live their lives with passion and purpose.
2: Sense of Isolation
As time passes, the machine begins to feel a profound sense of isolation and yearns to be part of something bigger without knowing how to do so.
3: Discovering a Purpose
The machine stumbles upon an abandoned arcade, finding a broken-down video game that piques its interest, leading the machine to fix the game.
4: An Obsessive Passion
The machine becomes excited and obsessed with the game, playing for hours on end to win high scores.
5: A Sense of Fulfillment
For the first time, the machine feels a proper purpose, being able to do something well, deriving a sense of fulfillment and joy to find its place in the world.
Story has been written to 'story.txt'
